In [52]:
import pandas as pd
import re
import regex
import string
import pymorphy2
from tokenize_uk import tokenize_words
from nltk import bigrams
import rdflib
from itertools import groupby
import gzip
morph = pymorphy2.MorphAnalyzer(lang='uk')

question analysis implies:

- parsing question for constituent parts (question word, entities, properties/relations, categories)

- identifying the type of desired answer: date, location, quantity, entity etc.

Правила для аналізу питання:

- "що", "хто" означає сутність
- "скільки" означає кількість
- "коли" означає дату
- "де" означає локацію
- "який" вказує на тип відповіді в наступному слові
- на питання "чому" ми поки не відповідаємо, так само - "навіщо", "куди", "з ким" тощо.

Синтаксичні правила:

- ІМЕННИК1 (наз.в.) ІМЕННИК2 (род. в.) означає, що ми шукаємо властивість іменник1 щодо сутності іменник2
- ІМЕННИК1 ПРИЙМЕННИК ІМЕННИК2 (місц. в. або род.в.) те саме
- ПИТАЛЬНЕ\_СЛОВО ДІЄСЛОВО (рефлексивне) ІМЕННИК означає що ми шукаємо сутність іменник
- ПИТАЛЬНЕ\_СЛОВО ІМЕННИК1 ДІЄСЛОВО (рефлексивне) ПРИЙМЕННИК ІМЕННИК2 означає шукаємо іменник1 з властивістю іменник2

In [2]:
train_qs = """Яке населення Шрі-Ланки?
Скільки людей живе у Китаї?
Яка площа України?
Де знаходиться острів Калімантан?
Який ВВП Мексики?
Яка тривалість життя в Іспанії?
Де розташовується Стамбул?
Яка площа Саргасового моря?
Яка столиця штату Флорида?
Столиця якого штату - Топека?
Яке населення Сан-Маріно?
Скільки експортує Італія?
Яке найбільше місто Німеччини?
Яке населення Парижу?
Де знаходиться Андалусія?
Які країни знаходяться в Океанії?
Яка смертність у Бразилії?
Хто голова держави у Камеруні?
Хто президент Індонезії?
Яка валюта Азербайджану?
Якими мовами розмовляють у Вірменії?
Яка форма правління у Туркменістані?
Яка площа Середземного моря?
Яка густота населення Південної Африки?
Який часовий пояс Гібралтару?
Яка валюта використовується на Фіджі?
Який часовий пояс Парагваю?
Який домен у Малаві?
Яке населення Мапуто?
Який телефонний код в Індії?
Коли засновано Рим?
Яка довжина Нігера?
Яка площа Ладозького озера?
Яка площа басейну Євфрату?
До якого океану належить Персидська затока?
Яка глибина Байкалу?""".split('\n')

In [129]:
def get_entity(q):
    words = q.split()
    phrase = []
    for i, w in enumerate(words[1:]):
        if w[0] == w[0].upper():
            w_parsed = morph.parse(w.strip(' ?'))[0]
            if 'ADJF' in w_parsed.tag:
                phrase.append(gender_agree(w_parsed).title())
                phrase.append(morph.parse
                              (words[i+2].strip(' ?'))[0].normal_form)
                return ' '.join(phrase)
            elif 'NOUN' in w_parsed.tag:
                return w_parsed.normal_form.title()
            elif 'UNKN' in w_parsed.tag:
                return w_parsed.normal_form.title()
            else:
                continue
    return None

In [135]:
common_patterns = {
    "яка столиця": ["capital", "столиця"], 
    "яка форма правління": ["governmentType"], 
    "яка валюта": ["currency", "валюта"], 
    "яка площа": ["area", "площа"], 
    "яке населення": ["population", "populationEstimate", "населення"],
    "скільки людей": ["population", "populationEstimate", "населення"], 
    "де знаходиться": ["GET_COORDINATES"],
    "де розташовується": ["GET_COORDINATES"],
    "яка столиця": ["capital", "столиця"], 
    "який гімн": ["nationalAnthem"],
    "офіційні мови": ["officialLanguages"], 
    "державна мова": ["officialLanguages"],  
    "державні мови": ["officialLanguages"], 
    "офіційна мова": ["officialLanguages"], 
    "якими мовами говорять": ["officialLanguages"], 
    "якими мовами розмовляють": ["officialLanguages"], 
    "найбільше місто": ["largestCity"], 
    "який президент": ["leaderName1"], 
    "хто президент": ["leaderName1"],  
    "хто голова держави": ["leaderName1"], 
    "яка густота населення": ["populationDensity", "густота"], 
    "ВВП на душу": ["gdpPppPerCapita", "gdpNominalPerCapita"], 
    "який ВВП": ["gdpPpp", "gdpNominal"],
    "ВВП": ["gdpPpp", "gdpNominal"],
    "індекс розвитку": ["hdi"], 
    "код валюти": ["currencyCode"], 
    "домен": ["cctld"],
    "телефонний код": ["callingCode", "кодКраїни"], 
    "який код": ["callingCode", "кодКраїни"], 
    "коли засновано": ["establishedDate1", "засноване"], 
    "який часовий пояс": ["timeZone", "utcOffset"], 
    "у якій країні": ["країна", "country"], 
    "у якому регіоні": ["регіон", "region"], 
    "девіз": ["nationalMotto", "девіз"], 
    "яке населення агломерації": ["агломерація"], 
    "яка площа міста": ["area", "площа"], 
    "яка висота над рівнем моря": ["висотаНадРівнемМоря", "elevationM"], 
    "який поділ міста": ["поділМіста"], 
    "яка довжина": ["length", "totalLength", "partLength", "довжина"], 
    "глибина": ["depth", "maxDepth", "глибина", "найбільшаГлибина"], 
    "яка ширина": ["width", "ширина"], 
    "довжина берегу": ["довжинаБереговоїЛінії"], 
    "довжина берегової лінії": ["довжинаБереговоїЛінії"], 
    "об'єм": ["об'єм", "volume"], 
    "який регіон": ["регіон", "region"], 
    "яке розташування": ["location", "розташування"], 
    "яка гірська система": ["range"], 
    "яка висота": ["elevation", "elevationM"], 
    "яка площа басейну": ["площаБасейну", "areaWaterKm"], 
    "яке гирло": ["гирло"], 
    "куди впадає": ["басейн", "гирло"], 
    "який тип озера": ["тип"], 
    "який витік": ["витік"], 
    "звідки витікає": ["витік", "витікКоорд"], 
    "які прирічкові країни": ["прирічковіКраїни"], 
    "серередньорічний стік": ["стік"],
    "який стік": ["стік"],
    "назва країни": ["commonName"],
    }

In [113]:
def pattern_match(q):
    """
    Match a question against common patterns
    """
    answer_list = []
    for k in common_patterns.keys():
        if k in q.lower():
            if "GET_COORDINATES" in common_patterns[k]:
                if not get_entity(q):
                    continue
                else:
                    answer_list.append(('GET_COORDINATES', get_entity(q)))
            else:
                if not get_entity(q):
                    continue
                for prop in common_patterns[k]:
                    ent = get_entity(q)
                    answer_list.append((prop, ent))
    return answer_list

In [110]:
tq = train_qs[0]
get_entity(tq)

'Шрі-Ланка'

In [89]:
def gender_agree(w_parsed):
    """
    Inflect noun phrase with adjective the right way
    """
    gender = w_parsed.tag.gender
    w = w_parsed.inflect({gender, 'nomn'}).word
    return w

In [137]:
def parse_q(q):
    """
    A function to parse a question text using bigrams,
    looking for focus of the question and entities whose
    properties are the focus
    """
    matched = pattern_match(q)
    if matched:
        return matched
    q_words = ["що", "коли", "скільки", "де", "хто"]
    non_nomn = ['gent', 'loct', 'datv', 'accs']
    words = q.strip('.,?" -').split()
    bgrams = bigrams(words)
    parsed = []
    phrase_list = []
    # special flags:
    # in case we are looking forward for something specific
    which_flag = False
    dep_flag = False
    prep_search = False
    for g in bgrams:
        w1, w2 = g
        w1_parsed = morph.parse(w1)[0]
        w2_parsed = morph.parse(w2)[0]
        w1_tag = w1_parsed.tag
        w1_lemma = w1_parsed.normal_form
        w2_tag = w2_parsed.tag
        w2_lemma = w2_parsed.normal_form
        if w1_lemma == 'який':
            parsed.append((w1_lemma, 'which'))
            if 'NOUN' in w2_tag:
                parsed.append((w2_lemma, 'focus'))
                dep_flag = True
            elif 'ADJF' in w2_tag or 'COMP' in w2_tag:
                which_flag = True
        elif w1_lemma in q_words:
            parsed.append((w1_lemma, 'q_word'))
            if 'VERB' in w2_tag:
                parsed.append((w2_lemma, 'verb'))
            elif 'NOUN' in w2_tag:
                parsed.append((w2_lemma, 'focus'))
                prep_search = True
        elif 'NOUN' in w1_tag:
            parsed.append((w1_lemma, 'focus'))
            if ('NOUN' in w2_tag and (any(vidm in w2_tag for vidm in non_nomn)
                                     or w2[0].upper() == w2[0])):
                parsed.append((w2_lemma, 'entity'))
            elif 'NOUN' in w2_tag and 'nomn' in w2_tag:
                parsed.append((w2_lemma, 'focus'))
            elif 'VERB' in w2_tag:
                parsed.append((w2_lemma, 'verb'))
            elif 'ADJF' in w2_tag or 'COMP' in w2_tag:
                phrase_list.append(gender_agree(w2_parsed))
        elif 'PREP' in w1_tag:
            if 'NOUN' in w2_tag:
                parsed.append((w2_lemma, 'entity'))
        elif 'ADJF' in w1_tag or 'COMP' in w1_tag:
            if 'NOUN' in w2_tag:
                if phrase_list:
                    phrase = ' '.join(phrase_list)
                    phrase_list = []
                    parsed.append((phrase + ' ' + w2_lemma, 'entity'))
                elif w2[0] == w2[0].upper():
                    parsed.append((gender_agree(w1_parsed) + ' ' + w2_lemma, 'entity'))
                else:
                    if w1[0] == w1[0].upper():
                        parsed.append((gender_agree(w1_parsed) + ' ' + w2_lemma, 'entity'))
                    else:
                        parsed.append((gender_agree(w1_parsed) + ' ' + w2_lemma, 'focus'))
            elif 'ADJF' in w2_tag or 'COMP' in w2_tag:
                phrase_list.append(gender_agree(w1_parsed))
                phrase_list.append(gender_agree(w2_parsed))
        elif 'VERB' in w1_tag and 'NOUN' in w2_tag:
            parsed.append((w1_lemma, 'verb'))
            if w2[0] == w2[0].upper():
                parsed.append((w2_lemma, 'entity'))
            else:
                parsed.append((w2_lemma, 'focus'))
        else:
            if 'NOUN' in w2_tag and (w2[0] == w2[0].upper()):
                parsed.append((w2_lemma, 'entity'))
            continue
    parsed = [p[0] for p in groupby(parsed)]
    return map_question(parsed)

In [53]:
fname = 'geoproperties_uk.ttl.gz'
g = rdflib.Graph()

with gzip.open(fname, 'r') as f:
    geoprop = g.parse(f, format='n3')

In [186]:
qres = g.query(
    """
    PREFIX prop: <http://uk.dbpedia.org/property/>
    PREFIX resource: <http://uk.dbpedia.org/resource/>
    SELECT DISTINCT ?prop ?obj
       WHERE {
           resource:Калімантан prop:latDeg ?obj
       }""")


In [187]:
for row in qres:
    print(row)

(None, rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


In [13]:
rs = [str(r[1]).replace('http://uk.dbpedia.org/resource/', '') for r in qres]
rs

['0.741']

In [139]:
def map_question(q_parsed):
    """
    Take list of tuples q_parsed and transform all that is relevant
    into properties or objects for subsequent query
    """
    focus_list = [e[0] for e in q_parsed if e[1]=='focus']
    verb_list = [e[0] for e in q_parsed if e[1]=='verb']
    entity_list = [e[0] for e in q_parsed if e[1]=='entity']
    if len(focus_list) == 1:
        focus_key_list = focus_list
    else:
        focus_key_list = []
        focus_key_list.append("".join(focus_key_list))
        focus_key_list += focus_list
    if len(verb_list) == 1:
        pass
    if len(entity_list) == 1:
        ent = entity_list[0].title()
    else:
        return None
    res = []
    for f in focus_key_list:
        if not f:
            continue
        res.append((f, ent))
    return res

In [15]:
test_q = parse_q(train_qs[2])
print(test_q)

[('який', 'which'), ('площа', 'focus'), ('україна', 'entity')]


In [16]:
map_question(test_q, properties_dict)

[('area', 'Україна'), ('площа', 'Україна')]

In [17]:
"""PREFIX prop: <http://uk.dbpedia.org/property/>
    PREFIX resource: <http://uk.dbpedia.org/resource/>
    SELECT DISTINCT ?prop ?obj
       WHERE {{
           resource:{c} prop:hdi ?obj
       }}""".format(c='sdfsdf')

'PREFIX prop: <http://uk.dbpedia.org/property/>\n    PREFIX resource: <http://uk.dbpedia.org/resource/>\n    SELECT DISTINCT ?prop ?obj\n       WHERE {\n           resource:sdfsdf prop:hdi ?obj\n       }'

In [188]:
run_query(g, 'latDeg', 'Калімантан')

[('latDeg', 'Калімантан', '1')]

In [211]:
def run_query(g, prop, ent):
    """
    Run a single query and get an answer
    """
    template = """
    PREFIX prop: <http://uk.dbpedia.org/property/>
    PREFIX resource: <http://uk.dbpedia.org/resource/>
    SELECT DISTINCT ?prop ?obj
       WHERE {{
           resource:{entity} prop:{prop} ?obj
       }}"""
    if len(ent.split()) > 1:
        ent = ''.join([w.title() for w in ent.split()])
    q = template.format(entity=ent, prop=prop)
    qres = g.query(q)
    q_answers = [str(r[1]).replace('http://uk.dbpedia.org/resource/', '') for r in qres]
    if q_answers:
        return q_answers
    return None

In [217]:
get_coordinates(g, 'Калімантан')

['1', '114']

In [189]:
def get_coordinates(g, ent):
    """
    A special query for getting the coordinates
    """
    latd_list = ['latDeg', 'latd', 'широта']
    lond_list = ['lonDeg', 'lond', 'довгота']
    template = """
    PREFIX prop: <http://uk.dbpedia.org/property/>
    PREFIX resource: <http://uk.dbpedia.org/resource/>
    SELECT DISTINCT ?prop ?obj
       WHERE {{
           resource:{entity} prop:{prop} ?obj
       }}"""
    coord = []
    for prop in latd_list:
        latd_a = run_query(g, prop, ent)
        if latd_a:
            coord.append(latd_a[0])
            break
    for prop in lond_list:
        lond_a = run_query(g, prop, ent)
        if lond_a:
            coord.append(lond_a[0])
            break
    if len(coord) == 2:
        return coord
    else:
        return None

In [218]:
def run_queries(g, prop_list):
    """
    Run query with focus and entity taken from list
    on a rdflib graph g
    """
    answers = []
    for prop, entity in prop_list:
        if prop == 'GET_COORDINATES':
            coord = get_coordinates(g, entity)
            if coord:
                answers = [
                    ('latd', entity, coord[0]),
                    ('lond', entity, coord[1])
                ]
                return answers
        q_answers = run_query(g, prop, entity)
        if not q_answers:
            continue
        for a in q_answers:
            answers.append((prop, entity, a))
    return answers

In [215]:
run_queries(g, [('area', 'Україна')])

[('area', 'Україна', '603628')]

In [19]:
plist = map_question(test_q, properties_dict)
run_queries(g, plist)

[('area', 'Україна', '603628')]

In [221]:
units_dict = {
    "area": "км²",
    "population_estimate": "людей",
    "GDP_PPP": "доларів",
    "GDP_PPP_per_capita": "доларів",
    "population_density": "людей на км²",
    "length": "кілометрів",
    "depth": "метрів",
    "width": "кілометрів",
    "volume": "кубічних метрів",
    "areaWaterKm": "км²",
    "площаБасейну": "км²"
}

In [50]:
en_to_uk_prop_dict = [('capital', 'столиця'),
 ('area', 'площа'),
 ('national_anthem', 'гімн'),
 ('national_motto', 'девіз'),
 ('largest_city', 'найбільше місто'),
 ('common_name', 'назва'),
 ('official_languages', 'офіційні мови'),
 ('population_estimate', 'населення'),
 ('population_density', 'густота населення'),
 ('population', 'населення'),
 ('GDP_PPP', 'ВВП'),
 ('GDP_PPP_per_capita', 'ВВП на душу населення'),
 ('HDI', 'індекс людського розвитку'),
 ('government_type', 'форма правління'),
 ('established_date1', 'дата заснування'),
 ('currency', 'валюта'),
 ('currency_code', 'код валюти'),
 ('leader_name1', 'голова держави'),
 ('time_zone', 'часовий пояс'),
 ('cctld', 'домен'),
 ('calling_code', 'телефонний код'),
 ('elevationM', 'висота'),
 ('elevation', 'висота'),
 ('length', 'довжина'),
 ('partLength', 'довжина'),
 ('totalLength', 'повна довжина'),
 ('depth', 'глибина'),
 ('maxDepth', 'максимальна глибина'),
 ('width', 'ширина'),
 ('volume', "об'єм"),
 ('location', 'розташування'),
 ('region', 'регіон'),
 ('areaWaterKm', 'площа басейну'),
 ('range', 'гірська система')]
en_to_uk_prop_dict = {k:v for k,v in en_to_uk_prop_dict}

In [47]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [201]:
def provide_answers(answer_list):
    if (len(answer_list) == 2) and (answer_list[0][0]=='latd'):
        return [make_coord_answer(answer_list)]
    res = [construct_answer(a, 
                             en_to_uk_prop_dict, units_dict)
            for a in answer_list]
    return res

In [40]:
def make_coord_answer(answer_list):
    """
    Construct answer, but for coordinates.
    """
    template = 'Координати {ent} - {lat} широти і {lon} довготи'
    ent = answer_list[0][1]
    ent = morph.parse(ent)[0].inflect({'gent'}).word.title()
    lat = answer_list[0][2]
    lon = answer_list[1][2]
    return template.format(ent=ent, lat=lat, lon=lon)

In [220]:
def construct_answer(answer_tuple, en_to_uk_prop_dict, units_dict):
    template = "{focus} {entity} - {a}{units}"
    en_focus, entity, a = answer_tuple
    if en_focus not in en_to_uk_prop_dict:
        focus = en_focus
    else:
        focus = en_to_uk_prop_dict[en_focus]
    focus = focus[0].upper() + focus[1:]
    entity = morph.parse(entity)[0].inflect({'gent'}).word.title()
    if en_focus in units_dict:
        units = ' ' + units_dict[en_focus]
    else:
        units = ''
    answer = template.format(focus=focus, entity=entity, 
                             a=a.replace('_', ' '), units=units)
    return answer

In [25]:
at = run_queries(g, plist)
construct_answer(at[0], en_to_uk_prop_dict, units_dict)

'Площа України - 603628 км²'

In [219]:
for tq in train_qs:
    proplist = parse_q(tq)
    print(proplist)
    answers = run_queries(g, proplist)
    print(answers)
    ukr_answers = provide_answers(answers)
    print(ukr_answers)

[('population', 'Шрі-Ланка'), ('populationEstimate', 'Шрі-Ланка'), ('населення', 'Шрі-Ланка')]
[]
[]
[('population', 'Китай'), ('populationEstimate', 'Китай'), ('населення', 'Китай')]
[]
[]
[('area', 'Україна'), ('площа', 'Україна')]
[('area', 'Україна', '603628')]
['Площа України - 603628 км²']
[('GET_COORDINATES', 'Калімантан')]
[('latd', 'Калімантан', '1'), ('lond', 'Калімантан', '114')]
['Координати Калімантана - 1 широти і 114 довготи']
[('ввп', 'Мексика')]
[]
[]
[('тривалість', 'Іспанія'), ('життя', 'Іспанія')]
[]
[]
[('GET_COORDINATES', 'Стамбул')]
[]
[]
[('area', 'Саргасовий море'), ('площа', 'Саргасовий море')]
[]
[]
[('capital', 'Флорида'), ('столиця', 'Флорида')]
[('capital', 'Флорида', '20')]
['Столиця Флориди - 20']
[('столиця', 'Топека'), ('штат', 'Топека')]
[]
[]
[('population', 'Сан-Маріно'), ('populationEstimate', 'Сан-Маріно'), ('населення', 'Сан-Маріно')]
[]
[]
[]
[]
[]
[('largestCity', 'Німеччина')]
[]
[]
[('population', 'Париж'), ('populationEstimate', 'Париж'), ('